In [1]:
%load_ext autoreload
%autoreload 2

from util_0704_ch import *
from myalgorithm_0801_1 import algorithm

from collections import Counter
import pickle

## 실험 내용

- 3 주문 번들이 묶인 형태 파악해서 번들 가능성 확인시 불필요한 탐색 조합을 발견하기

### 테스트케이스 하나 실험

In [2]:
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_5.json'
problem_file = '../alg_test_problems_20240429/TEST_K200_2.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

for v in ALL_RIDERS:
    print(v)

print(K)

Rider([BIKE, 5.291005291005291, 100, 60, 5000, 120, 40])
Rider([WALK, 1.3227513227513228, 70, 30, 5000, 120, 60])
Rider([CAR, 4.2328042328042335, 200, 100, 5000, 180, 200])
200


In [3]:
## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

## ------------------  크루스칼 함수   -----------------------------

def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            if avg_method == 'avg':
                dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
            elif avg_method == 'two_seq':
                dist1 = DIST[i][j]
                dist2 = DIST[i + K][j + K]
            elif avg_method == 'two':
                order_num1 = all_bundles[i].shop_seq[0]
                order_num2 = all_bundles[j].shop_seq[0]

                dist1 = DIST[order_num1][order_num2]
                dist2 = DIST[order_num1 + K][order_num2 + K]  
            else:
                assert False

            # weight1 = (dist1 + dist2) / 900

            # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2
            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(len(all_bundles)))
    edges.sort(key=lambda x: x[2])

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue

        new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]
    rider_availables = [rider.available_number for rider in ALL_RIDERS]

    return result_bundles, rider_availables

## --------------- 초기 번들링 최적화 코드 --------------------------

weight1 = 1
weight2 = -1

avg_method = 'two'
bundle_merging_function = try_merging_bundles_by_dist
default_get_dist_function = get_dist_by_coords

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

# print('#2\n')

# 2개 주문 묶음 생성
all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

# print('#4\n')

# 4개 주문 묶음 생성
all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

# 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
new_all_bundles = []
for bundle in all_bundles:
    if len(bundle.shop_seq) >= 3:
        new_all_bundles.append(bundle)
    else:
        old_rider = bundle.rider
        old_rider.available_number += 1
        for order_num in bundle.shop_seq:
            order = ALL_ORDERS[order_num]

            new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
            car_rider.available_number -= 1
            new_all_bundles.append(new_bundle)

# print('#remain\n')

all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

## ------------------- 라이더 재배치 -------------------------------

all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

checked_solution['avg_cost']

3310.4569999999994

In [7]:
three_order_bundles = [bundle for bundle in checked_solution['bundles'] if len(bundle[1]) == 3]

ready_time_cts = [Counter() for _ in range(3)]
deadline_cts = [Counter() for _ in range(3)]
for bundle in three_order_bundles:
    ready_times = []
    deadlines = []
    for order_num in bundle[1]:
        order = ALL_ORDERS[order_num]

        ready_times.append((order_num, order.ready_time))
        deadlines.append((order_num, order.deadline))

    ready_times.sort(key=lambda x: x[1])
    deadlines.sort(key=lambda x: x[1])

    ready_time_nths = [0] * 3
    deadline_nths = [0] * 3

    shop_seq = bundle[1]
    dlv_seq = bundle[2]
    for nth, (order_num, _) in enumerate(ready_times):
        shop_index = shop_seq.index(order_num)
        ready_time_nths[shop_index] = nth + 1

        dlv_index = dlv_seq.index(order_num)
        deadline_nths[dlv_index] = nth + 1

    for i in range(3):
        ready_time_cts[i][ready_time_nths[i]] += 1
        deadline_cts[i][deadline_nths[i]] += 1

    for i in range(3):
        for nth in range(1, 4):
            ready_time_cts[i][nth] += 0
            deadline_cts[i][nth] += 0

ready_time_freqs = [sorted(ct.items(), key=lambda x: x[0]) for ct in ready_time_cts]
deadline_freqs = [sorted(ct.items(), key=lambda x: x[0]) for ct in deadline_cts]

ready_time_freqs

[[(1, 17), (2, 9), (3, 3)],
 [(1, 8), (2, 14), (3, 7)],
 [(1, 4), (2, 6), (3, 19)]]

### 모든 테스트 케이스 대상으로 실험

In [8]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    problem_file_name = problem_file.split('\\')[-1]

    ## ------------------- 초기 상태 할당 코드 -------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    ## ------------------  크루스칼 함수   -----------------------------

    def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
        def find(v):
            while v != parent[v]:
                parent[v] = parent[parent[v]]
                v = parent[v]

            return v

        def union(a, b, new_bundle):
            if a > b:
                a, b = b, a

            parent[b] = a
            all_bundles[a] = new_bundle

        for i in range(len(all_bundles)):
            bundle = all_bundles[i]

            shop_seq = bundle.shop_seq

            xs_s_sum = 0
            ys_s_sum = 0

            xs_e_sum = 0
            ys_e_sum = 0

            readytimes_sum = 0
            deadlines_sum = 0

            shop_seq_len = len(shop_seq)

            for order_num in shop_seq:
                order = ALL_ORDERS[order_num]

                xs_s_sum += order.shop_lat
                ys_s_sum += order.shop_lon

                xs_e_sum += order.dlv_lat
                ys_e_sum += order.dlv_lon

                readytimes_sum += order.ready_time
                deadlines_sum += order.deadline

            xs_s_avg = xs_s_sum / shop_seq_len
            ys_s_avg = ys_s_sum / shop_seq_len

            xs_e_avg = xs_e_sum / shop_seq_len
            ys_e_avg = ys_e_sum / shop_seq_len

            readytimes_avg = readytimes_sum / shop_seq_len
            deadlines_avg = deadlines_sum / shop_seq_len

            avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

            bundle.avg_info = avg_info

        edges = []
        for i in range(len(all_bundles)):
            for j in range(i + 1, len(all_bundles)):
                avg_info1 = all_bundles[i].avg_info
                avg_info2 = all_bundles[j].avg_info

                sx1, sy1, ex1, ey1, r1, d1 = avg_info1
                sx2, sy2, ex2, ey2, r2, d2 = avg_info2

                r_diff = abs(r1 - r2)
                d_diff = abs(d1 - d2)

                start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

                if avg_method == 'avg':
                    dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                    dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
                elif avg_method == 'two_seq':
                    dist1 = DIST[i][j]
                    dist2 = DIST[i + K][j + K]
                elif avg_method == 'two':
                    order_num1 = all_bundles[i].shop_seq[0]
                    order_num2 = all_bundles[j].shop_seq[0]

                    dist1 = DIST[order_num1][order_num2]
                    dist2 = DIST[order_num1 + K][order_num2 + K]  
                else:
                    assert False

                # weight1 = (dist1 + dist2) / 900

                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2
                diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

                edges.append((i, j, diff_score))

        parent = list(range(len(all_bundles)))
        edges.sort(key=lambda x: x[2])

        for bundle_num1, bundle_num2, diff_score in edges:
            rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

            if rbn1 == rbn2:
                continue

            new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

            if new_bundle is not None:
                all_bundles[rbn1].rider.available_number += 1
                all_bundles[rbn2].rider.available_number += 1
                
                new_bundle.rider.available_number -= 1

                union(rbn1, rbn2, new_bundle)

        parent = [find(v) for v in parent]

        result_bundles = [all_bundles[v] for v in set(parent)]
        rider_availables = [rider.available_number for rider in ALL_RIDERS]

        return result_bundles, rider_availables

    ## --------------- 초기 번들링 최적화 코드 --------------------------

    weight1 = 1
    weight2 = 0

    avg_method = 'two'
    bundle_merging_function = try_merging_bundles_by_dist
    default_get_dist_function = get_dist_by_coords

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    # print('#2\n')

    # 2개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

    # print('#4\n')

    # 4개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

    # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
    new_all_bundles = []
    for bundle in all_bundles:
        if len(bundle.shop_seq) >= 3:
            new_all_bundles.append(bundle)
        else:
            old_rider = bundle.rider
            old_rider.available_number += 1
            for order_num in bundle.shop_seq:
                order = ALL_ORDERS[order_num]

                new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                car_rider.available_number -= 1
                new_all_bundles.append(new_bundle)

    # print('#remain\n')

    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

    ## ------------------- 라이더 재배치 -------------------------------

    all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

    ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

    solution = [
            # rider type, shop_seq, dlv_seq
            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
            for bundle in all_bundles
    ]

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    three_order_bundles = [bundle for bundle in checked_solution['bundles'] if len(bundle[1]) == 3]

    ready_time_cts = [Counter() for _ in range(3)]
    deadline_cts = [Counter() for _ in range(3)]
    for bundle in three_order_bundles:
        ready_times = []
        deadlines = []
        for order_num in bundle[1]:
            order = ALL_ORDERS[order_num]

            ready_times.append((order_num, order.ready_time))
            deadlines.append((order_num, order.deadline))

        ready_times.sort(key=lambda x: x[1])
        deadlines.sort(key=lambda x: x[1])

        ready_time_nths = [0] * 3
        deadline_nths = [0] * 3

        shop_seq = bundle[1]
        dlv_seq = bundle[2]
        for nth, (order_num, _) in enumerate(ready_times):
            shop_index = shop_seq.index(order_num)
            ready_time_nths[shop_index] = nth + 1

            dlv_index = dlv_seq.index(order_num)
            deadline_nths[dlv_index] = nth + 1

        for i in range(3):
            ready_time_cts[i][ready_time_nths[i]] += 1
            deadline_cts[i][deadline_nths[i]] += 1

        for i in range(3):
            for nth in range(1, 4):
                ready_time_cts[i][nth] += 0
                deadline_cts[i][nth] += 0

    ready_time_freqs = [sorted(ct.items(), key=lambda x: x[0]) for ct in ready_time_cts]
    deadline_freqs = [sorted(ct.items(), key=lambda x: x[0]) for ct in deadline_cts]

    results.append((problem_file_name, K, ready_time_freqs, deadline_freqs))

    print(problem_file_name)
    

TEST_K50_1.json
TEST_K50_2.json
TEST_K100_1.json
TEST_K100_2.json
TEST_K200_1.json
TEST_K200_2.json
STAGE1_1.json
STAGE1_2.json
STAGE1_3.json
STAGE1_4.json
STAGE1_5.json
STAGE1_6.json
STAGE1_7.json
STAGE1_8.json
STAGE1_9.json
STAGE1_10.json
STAGE1_11.json
STAGE1_12.json
STAGE1_13.json
STAGE1_14.json
STAGE1_15.json
STAGE1_16.json
STAGE1_17.json
STAGE1_18.json


#### 결과 분석

In [11]:
[[v[0], v[1], v[2]] for v in results]

[['TEST_K50_1.json',
  50,
  [[(1, 3), (2, 5), (3, 1)],
   [(1, 3), (2, 2), (3, 4)],
   [(1, 3), (2, 2), (3, 4)]]],
 ['TEST_K50_2.json',
  50,
  [[(1, 4), (2, 3), (3, 1)],
   [(1, 4), (2, 4), (3, 0)],
   [(1, 0), (2, 1), (3, 7)]]],
 ['TEST_K100_1.json',
  100,
  [[(1, 7), (2, 5), (3, 1)],
   [(1, 3), (2, 4), (3, 6)],
   [(1, 3), (2, 4), (3, 6)]]],
 ['TEST_K100_2.json',
  100,
  [[(1, 13), (2, 3), (3, 1)],
   [(1, 3), (2, 8), (3, 6)],
   [(1, 1), (2, 6), (3, 10)]]],
 ['TEST_K200_1.json',
  200,
  [[(1, 22), (2, 7), (3, 2)],
   [(1, 7), (2, 14), (3, 10)],
   [(1, 2), (2, 10), (3, 19)]]],
 ['TEST_K200_2.json',
  200,
  [[(1, 19), (2, 7), (3, 4)],
   [(1, 4), (2, 17), (3, 9)],
   [(1, 7), (2, 6), (3, 17)]]],
 ['STAGE1_1.json',
  100,
  [[(1, 5), (2, 7), (3, 1)],
   [(1, 3), (2, 5), (3, 5)],
   [(1, 5), (2, 1), (3, 7)]]],
 ['STAGE1_2.json',
  100,
  [[(1, 2), (2, 3), (3, 3)],
   [(1, 2), (2, 1), (3, 5)],
   [(1, 4), (2, 4), (3, 0)]]],
 ['STAGE1_3.json',
  200,
  [[(1, 16), (2, 6), (3, 1)],


In [12]:
[[v[0], v[1], v[3]] for v in results]

[['TEST_K50_1.json',
  50,
  [[(1, 6), (2, 1), (3, 2)],
   [(1, 2), (2, 5), (3, 2)],
   [(1, 1), (2, 3), (3, 5)]]],
 ['TEST_K50_2.json',
  50,
  [[(1, 3), (2, 3), (3, 2)],
   [(1, 3), (2, 4), (3, 1)],
   [(1, 2), (2, 1), (3, 5)]]],
 ['TEST_K100_1.json',
  100,
  [[(1, 6), (2, 2), (3, 5)],
   [(1, 3), (2, 7), (3, 3)],
   [(1, 4), (2, 4), (3, 5)]]],
 ['TEST_K100_2.json',
  100,
  [[(1, 6), (2, 4), (3, 7)],
   [(1, 9), (2, 5), (3, 3)],
   [(1, 2), (2, 8), (3, 7)]]],
 ['TEST_K200_1.json',
  200,
  [[(1, 11), (2, 14), (3, 6)],
   [(1, 14), (2, 11), (3, 6)],
   [(1, 6), (2, 6), (3, 19)]]],
 ['TEST_K200_2.json',
  200,
  [[(1, 16), (2, 8), (3, 6)],
   [(1, 8), (2, 15), (3, 7)],
   [(1, 6), (2, 7), (3, 17)]]],
 ['STAGE1_1.json',
  100,
  [[(1, 6), (2, 6), (3, 1)],
   [(1, 5), (2, 3), (3, 5)],
   [(1, 2), (2, 4), (3, 7)]]],
 ['STAGE1_2.json',
  100,
  [[(1, 4), (2, 2), (3, 2)],
   [(1, 4), (2, 1), (3, 3)],
   [(1, 0), (2, 5), (3, 3)]]],
 ['STAGE1_3.json',
  200,
  [[(1, 10), (2, 7), (3, 6)],
  

In [13]:
from itertools import permutations

li = [4, 3, 2, 1]
for case in permutations(li):
    print(case)

(4, 3, 2, 1)
(4, 3, 1, 2)
(4, 2, 3, 1)
(4, 2, 1, 3)
(4, 1, 3, 2)
(4, 1, 2, 3)
(3, 4, 2, 1)
(3, 4, 1, 2)
(3, 2, 4, 1)
(3, 2, 1, 4)
(3, 1, 4, 2)
(3, 1, 2, 4)
(2, 4, 3, 1)
(2, 4, 1, 3)
(2, 3, 4, 1)
(2, 3, 1, 4)
(2, 1, 4, 3)
(2, 1, 3, 4)
(1, 4, 3, 2)
(1, 4, 2, 3)
(1, 3, 4, 2)
(1, 3, 2, 4)
(1, 2, 4, 3)
(1, 2, 3, 4)


In [ ]:
case 